In [1]:
import pyodbc 
import time
import pandas as pd
from IPython.display import display

### Get an ODBC connection 

In [2]:
dsn = 'IRIS QuickML demo via PyODBC'
server = 'irisimlsvr' #'192.168.99.101' 
port = '51773' #'9091'
database = 'USER' 
username = 'SUPERUSER' 
password = 'SYS' 
cnxn = pyodbc.connect('DRIVER={InterSystems ODBC35};SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

### Ensure it read strings correctly.
cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf8')
cnxn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf8')
cnxn.setencoding(encoding='utf8')

### Get a cursor; start the timer

In [3]:
cursor = cnxn.cursor()
start= time.clock()

### Model configuration

In [4]:
dataTable = 'Passengers'
dataColumn =  'Survived'
modelName = "SurviveClassification"
trainProportion = ".8"

### Train and predict

In [5]:
df1 = pd.read_sql(
    "SELECT COUNT(*) AS ModelExists FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS WHERE MODEL_NAME = '%s'"
    % (modelName), 
    cnxn
)
if (df1['ModelExists'][0] == 1):
    print('Deleting previous model...')
    cursor.execute("DROP MODEL %s" % (modelName))

In [6]:
cursor.execute("CREATE MODEL %s PREDICTING (%s)  FROM %s" % (modelName, dataColumn, dataTable))
cursor.execute("TRAIN MODEL %s FROM %s WHERE ID < (SELECT COUNT(*) * %s FROM %s)" % (modelName, dataTable, trainProportion, dataTable))
cnxn.commit()

In [7]:
cursor.execute("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS")
df1 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", cnxn)
display(df1)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,bc,bc2,AutoML,2020-07-08 11:25:26.578,classification,"ModelType:Logistic Regression, Package:sklearn..."
1,SurviveClassification,SurviveClassification2,AutoML,2020-07-08 11:27:08.419,classification,"ModelType:Random Forest, Package:sklearn, Prob..."


### Show the predict result

In [8]:
df1 = pd.read_sql(
    "SELECT PREDICT(%s) As Predicted, Survived FROM %s WHERE ID > (SELECT COUNT(*) * %s FROM %s)" 
    % (modelName, dataTable, trainProportion, dataTable), 
    cnxn
)
display(df1)

,Predicted,Survived
0,0,1
1,0,0
2,1,0
3,0,0
4,1,1
...,...,...
174,0,0
175,1,1
176,0,0
177,1,1


### VALIDATE MODEL command calculates accuracy metrics

In [9]:
cursor.execute("VALIDATE MODEL %s FROM %s" % (modelName, dataTable))
df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS WHERE MODEL_NAME = '%s'" % (modelName), cnxn)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

METRIC_NAME,Accuracy,F-Measure,Precision,Recall
VALIDATION_RUN_NAME,,,,
SurviveClassification21,0.97,0.96,0.97,0.96


### Close and clean 

In [10]:
cnxn.close()
end= time.clock()
print ("Total elapsed time: ")
print (end-start)

Total elapsed time: 
0.1289929999999999


In [23]:
!pip install /tmp/irisnative-1.2.0-cp34-abi3-linux_x86_64.whl

Processing /tmp/irisnative-1.2.0-cp34-abi3-linux_x86_64.whl


In [24]:
import irisnative

In [86]:
ip = "irisimlsvr"
port = 51773
namespace = "USER"
username = "SUPERUSER"
password = "SYS"

# create database connection and IRIS instance
connection = irisnative.createConnection(ip, port, namespace, username, password)
dbnative = irisnative.createIris(connection)

In [87]:
print(dbnative.classMethodValue("MyMetric.Install", "Test", "MyMetric.IntegratedMLModelsValidation"))

MyMetric - IntegratedMLModelsValidation   AppointmentsShowPredection2655694605153933   Serial value of object or the object identity:TMyMetric||IntegratedMLModelsValidation||AppointmentsShowPredection2655694605153933.MyMetric.Sample.IntegratedMLModelsValidation
   Accuracy:                   .81
   F-Measure:                   .9
   Precision:                  .81
   Recall:                       1
   Name of the model definition:APPOINTMENTSSHOWPREDECTION
   Name of the trained model being validated:AppointmentsShowPredection2655694605153933
   Validation error (if encountered):1




In [88]:
connection.close()